In [ ]:
import os
import random
import numpy as np
import sys
import torch
import pprint

root = '/'
pprint.pprint(sys.path)

sys.path.append(import_path)
pprint.pprint(sys.path)

In [ ]:
import os
import sys
import pprint

import_path = root + 'pigvae_all'
sys.path.append(import_path)
pprint.pprint(sys.path)

import_path1 = root + 'pigvae_all/pigvae'
sys.path.append(import_path1)
pprint.pprint(sys.path)

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data.distributed import DistributedSampler
import random
#import pytorch_lightning as pl
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
from networkx.algorithms.shortest_paths.dense import floyd_warshall_numpy

from networkx.generators.random_graphs import *
from networkx.generators.ego import ego_graph
from networkx.generators.geometric import random_geometric_graph

/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred wh

In [ ]:
loaded_data = torch.load(root + 'dataset/train_dataset/zinc_gdata/tensor_data.pkl')

# 各テンソルにアクセス
node_features = loaded_data['node_features']
edge_features = loaded_data['edge_features']
mask = loaded_data['mask']
props = loaded_data['props']

<ipython-input-8-9f298e5238d8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(root + 'zinc_gdata/tensor_data.pkl')


In [ ]:
n_train = 50000
n_val = 60000
n_test = 100000
n_batch = 20

train_dataset = torch.utils.data.TensorDataset(node_features[0:n_train], edge_features[0:n_train], mask[0:n_train], props[0:n_train])
val_dataset = torch.utils.data.TensorDataset(node_features[n_train:n_val], edge_features[n_train:n_val], mask[n_train:n_val], props[n_train:n_val])
test_dataset = torch.utils.data.TensorDataset(node_features[n_val:n_test], edge_features[n_val:n_test], mask[n_val:n_test], props[n_val:n_test])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=n_batch, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=n_batch, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=n_batch, shuffle=False)

In [ ]:
from pigvae.models import GraphAE

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_node_f, num_edge_f = 43, 6

In [ ]:
hparams = {
    "vae":True,
    "kld_loss_scale":0.01,
    "perm_loss_scale":0.1,
    "property_loss_scale":0.5,
    "num_node_features":num_node_f,
    "num_edge_features":6+num_edge_f+1,
    "emb_dim": 90,
    'graph_encoder_hidden_dim': 256,
    'graph_encoder_k_dim': 64,
    'graph_encoder_v_dim': 64,
    'graph_encoder_num_heads': 16,
    'graph_encoder_ppf_hidden_dim': 512,
    'graph_encoder_num_layers': 16,
    'graph_decoder_hidden_dim': 256,
    'graph_decoder_k_dim': 64,
    'graph_decoder_v_dim': 64,
    'graph_decoder_num_heads': 16,
    'graph_decoder_ppf_hidden_dim': 512,
    'graph_decoder_num_layers': 16,
    "graph_decoder_pos_emb_dim": 64,
    'property_predictor_hidden_dim': 3,
    'num_properties': 1
}


model_save_dir = root + 'save_models/zinc/pig-beta-nflow-vae_models/'

if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as distrib
import torch.distributions.transforms as transform

class Flow(transform.Transform, nn.Module):

    def __init__(self):
        transform.Transform.__init__(self)
        nn.Module.__init__(self)

    # Init all parameters
    def init_parameters(self):
        for param in self.parameters():
            param.data.uniform_(-0.01, 0.01)

    # Hacky hash bypass
    def __hash__(self):
        return nn.Module.__hash__(self)

class PlanarFlow(Flow):

    def __init__(self, dim):
        super(PlanarFlow, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(1, dim))
        self.scale = nn.Parameter(torch.Tensor(1, dim))
        self.bias = nn.Parameter(torch.Tensor(1))
        self.init_parameters()

    def _call(self, z):
        f_z = F.linear(z, self.weight, self.bias)
        return z + self.scale * torch.tanh(f_z)

    def log_abs_det_jacobian(self, z):
        f_z = F.linear(z, self.weight, self.bias)
        psi = (1 - torch.tanh(f_z) ** 2) * self.weight
        det_grad = 1 + torch.mm(psi, self.scale.t())
        return torch.log(det_grad.abs() + 1e-9)

In [ ]:
# Main class for normalizing flow
class NormalizingFlow(nn.Module):

    def __init__(self, dim, blocks, flow_length, density):
        super().__init__()
        biject = []
        for f in range(flow_length):
            for b_flow in blocks:
                biject.append(b_flow(dim))
        self.transforms = transform.ComposeTransform(biject)
        self.bijectors = nn.ModuleList(biject)
        self.base_density = density
        #self.final_density = distrib.TransformedDistribution(density, self.transforms)
        self.log_det = []

    def forward(self, z):
        self.log_det = []
        # Applies series of flows
        for b in range(len(self.bijectors)):
            self.log_det.append(self.bijectors[b].log_abs_det_jacobian(z))
            z = self.bijectors[b](z)

        return z, self.log_det

In [ ]:
from pigvae.models import GraphEncoder, GraphDecoder, Permuter
from pigvae.models import BottleNeckEncoder, BottleNeckDecoder, PropertyPredictor

import torch
from torch.nn import Linear, LayerNorm, Dropout
from torch.nn.functional import relu, pad
from pigvae.graph_transformer import Transformer, PositionalEncoding
from pigvae.synthetic_graphs.data import DenseGraphBatch


import torch
from torch.nn import Linear, LayerNorm, Dropout
from torch.nn.functional import relu, pad
from pigvae.graph_transformer import Transformer, PositionalEncoding
#from pigvae.synthetic_graphs.data import DenseGraphBatch

block_planar = [PlanarFlow]

class GraphFlowVAE(torch.nn.Module):
    def __init__(self, hparams):
        super().__init__()
        self.vae = hparams["vae"]
        self.encoder = GraphEncoder(hparams)
        self.bottle_neck_encoder = BottleNeckEncoder(hparams)
        self.bottle_neck_decoder = BottleNeckDecoder(hparams)
        self.property_predictor = PropertyPredictor(hparams)
        self.permuter = Permuter(hparams)
        self.decoder = GraphDecoder(hparams)
        self.flow = NormalizingFlow(dim=hparams["emb_dim"], blocks=block_planar, flow_length=16, \
                               density=distrib.MultivariateNormal(torch.zeros(hparams["emb_dim"]), torch.eye(hparams["emb_dim"])))

    def encode(self, node_features, edge_features, mask):

        graph_emb, node_features = self.encoder(
            node_features=node_features,
            edge_features=edge_features,
            mask=mask,
        )
        graph_emb, mu, logvar = self.bottle_neck_encoder(graph_emb)

        return graph_emb, node_features, mu, logvar

    def decode(self, graph_emb, perm, mask=None):
        props = self.property_predictor(graph_emb).squeeze()

        graph_emb = self.bottle_neck_decoder(graph_emb)
        node_logits, edge_logits = self.decoder(
            graph_emb=graph_emb,
            perm=perm,
            mask=mask
        )

        return node_logits, edge_logits, props

    def forward(self, node_features, edge_features, mask, training, tau):

        #sampling z0 with VAE encoder
        z_0, node_features, mu, logvar = self.encode(node_features, edge_features, mask)
        perm = self.permuter(node_features, mask=mask, hard=not training, tau=tau)

        # sampling posterior  zk with flows
        z_k, list_ladj = self.flow(z_0)
        #Cross entropy ln p(z_k)
        log_p_zk = -0.5 * z_k * z_k
        #Entropy ln q(z_0)
        log_q_z0 = - torch.mean(torch.sum(logvar,dim=1)) * 1e-3
        #  ln q(z_0) - ln p(z_k)
        logs = (- log_p_zk).sum()
        # Add log determinants
        ladj = torch.cat(list_ladj)
        # ln q(z_0) - ln p(z_k) - sum[log det]
        logs -= torch.sum(ladj) #all flow loss

        nflow_loss = (logs / float(n_batch)) + log_q_z0
        graph_pred = self.decode(z_k, perm, mask)

        return graph_pred, perm, z_0, z_k, list_ladj, nflow_loss, mu, logvar


class BottleNeckEncoder(torch.nn.Module):
    def __init__(self, hparams):
        super().__init__()
        self.d_in = hparams["graph_encoder_hidden_dim"]
        self.d_out = hparams["emb_dim"]
        self.vae = hparams["vae"]
        if self.vae:
            self.w = Linear(self.d_in, 2 * self.d_out)
        else:
            self.w = Linear(self.d_in, self.d_out)

    def forward(self, x):
        x = self.w(relu(x))
        if self.vae:
            mu = x[:, :self.d_out]
            logvar = x[:, self.d_out:]
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            x = mu + eps * std
            return x, mu, logvar
        else:
            return x, None, None

In [ ]:
model = GraphFlowVAE(hparams).to(device)

In [ ]:
from pigvae.synthetic_graphs.critic_zinc import Critic

critic = Critic(hparams, device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, betas=(0.9, 0.98))
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.9)

# Training

In [ ]:
train_loss_list = []
val_loss_list = []

def val_preds(epoch):

    model.eval()

    sum_vae_errors = 0

    for batch_idx, batch_data in enumerate(val_loader):

        with torch.no_grad():

            if torch.cuda.is_available():
                node_features, edge_features, mask, props = batch_data
                node_features, edge_features, mask, props = node_features.to(device), edge_features.to(device), mask.to(device), props.to(device)

            #graph_pred, perm, mu, logvar = model(node_features, edge_features, mask, training=True, tau=1.0)
            graph_pred, perm, z_0, z_k, log_dets, nflow_loss, mu, logvar = model(node_features, edge_features, mask, training=True, tau=1.0)
            node_logits, edge_logits, pred_props = graph_pred

            vae_loss = critic(node_features, edge_features, props, node_logits, edge_logits, pred_props, mask, perm, mu, logvar)
            batch_all_loss = vae_loss['node_loss'] * 10 + vae_loss['edge_loss'] * 10 + vae_loss['perm_loss'] * 0.1 + nflow_loss * 0.1

            if batch_idx % 10 == 0:
                print("Epoch (val) : ", epoch, " batch (val) : ", batch_idx , "Loss (val) : ", batch_all_loss.data.item())

            sum_vae_errors = sum_vae_errors + vae_loss['node_loss'].data.item() * 10 + vae_loss['edge_loss'].data.item() * 10 + nflow_loss.data.item() * 0.1

            del batch_all_loss
            del vae_loss
            torch.cuda.empty_cache()

    return sum_vae_errors / len(val_loader)

In [ ]:
import datetime

# テキストファイルのパスを指定
log_path = model_save_dir + 'log.txt'

f = open(log_path, 'w')
f.write('train -- logs')
f.close()

def logging_txt(epoch):
    dt_now = datetime.datetime.now()

    # ファイルを読み込んで、末尾の行を取得
    with open(log_path, 'r') as file:
        lines = file.readlines()
        last_line = lines[-1].strip()

    # 末尾の行に追記する文字列を指定
    append_string = "epoch " + str(epoch) + " : " + str(dt_now)

    # ファイルに追記
    with open(log_path, 'a') as file:
        file.write('\n' + append_string)

In [ ]:
num_epochs = 100

best_loss = 10000000


for epoch in range(num_epochs):
    model.train()

    sum_vae_errors = 0

    for batch_idx, batch_data in enumerate(train_loader):

        node_features, edge_features, mask, props = batch_data
        node_features, edge_features, mask, props = node_features.to(device), edge_features.to(device), mask.to(device), props.to(device)

        optimizer.zero_grad()

        graph_pred, perm, z_0, z_k, log_dets, nflow_loss, mu, logvar = model(node_features, edge_features, mask, training=True, tau=1.0)
        node_logits, edge_logits, pred_props = graph_pred

        vae_loss = critic(node_features, edge_features, props, node_logits, edge_logits, pred_props, mask, perm, mu, logvar)
        batch_all_loss = vae_loss['node_loss'] * 10 + vae_loss['edge_loss'] * 10 + vae_loss['perm_loss'] * 0.1 + nflow_loss * 0.1

        if batch_idx % 100 == 0:
            print("Epoch : ", epoch, " Batch : ", batch_idx, " Loss : ", batch_all_loss.data.item())
            print(vae_loss)

        sum_vae_errors = sum_vae_errors + vae_loss['node_loss'].data.item() * 10 + nflow_loss.data.item() * 0.1
        batch_all_loss.backward()

        del batch_all_loss
        del vae_loss
        torch.cuda.empty_cache()

        optimizer.step()

    lr_scheduler.step()
    train_loss_list.append(sum_vae_errors / len(train_loader))

    val_loss = val_preds(epoch)
    val_loss_list.append(val_loss)

    np.save(model_save_dir + "train_loss_hist", np.array(train_loss_list))
    np.save(model_save_dir + "val_loss_hist", np.array(val_loss_list))

    logging_txt(epoch)

    if val_loss < best_loss:
        save_model_path = model_save_dir + "pigvae_best_model" + ".pt"
        torch.save(model, save_model_path)

        optimizer_path = model_save_dir + "opt_best_model" + ".pt"
        torch.save(optimizer.state_dict(), optimizer_path)

        best_loss = val_loss

Streaming output truncated to the last 5000 lines.
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(548.2404, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(2.9089, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(525.5051, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  91  Batch :  1960  Loss :  -4.826415061950684
{'edge_loss': tensor(0.0642, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0043, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(551.1786, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(2.8158, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(507.4319, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  91  Batch :  1970  Loss :  -4.8364763259887695
{'edge_loss': tensor(0.0643, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0036, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBack